import libraries

In [15]:
pip install scikit-learn
pip install pandas
pip install spacy
pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 16.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.8/29.8 MB 11.2 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
#import libraries
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import spacy

[nltk_data] Downloading package punkt to /Users/henry/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/henry/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


load dataset

In [3]:
# Load the dataset
data = pd.read_csv('../IT1244 Project/Twitter Sentiment/dataset.csv', header=None)
data.columns = ['labels', 'text']
print(len(data))
print(data.head())

100000
   labels                                               text
0       0  @switchfoot http://twitpic.com/2y1zl - Awww, t...
1       0  is upset that he can't update his Facebook by ...
2       0  @Kenichan I dived many times for the ball. Man...
3       0    my whole body feels itchy and like its on fire 
4       0  @nationwideclass no, it's not behaving at all....


Data preprocessing

We clean the data by removing url mentions hashtags and keeping only the letters then convert to lower case

Then we tokenise the text then we remove stop words and lanel the data

In [4]:
# Clean and preprocess the text data
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'@[A-Za-z0-9]+', '', text)  # Remove mentions
    text = re.sub(r'#', '', text)  # Remove hashtags
    text = re.sub(r'[^A-Za-z]+', ' ', text)  # Keep only letters
    text = text.lower()  # Convert to lowercase
    return text

data['cleaned_text'] = data['text'].apply(clean_text)

# Tokenize the text
data['tokens'] = data['cleaned_text'].apply(word_tokenize)

# Remove stop words
stop_words = set(stopwords.words('english'))
data['filtered_tokens'] = data['tokens'].apply(lambda tokens: [word for word in tokens if word not in stop_words])

data.head()

,labels,text,cleaned_text,tokens,filtered_tokens
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",awww that s a bummer you shoulda got david ca...,"[awww, that, s, a, bummer, you, shoulda, got, ...","[awww, bummer, shoulda, got, david, carr, thir..."
1,0,is upset that he can't update his Facebook by ...,is upset that he can t update his facebook by ...,"[is, upset, that, he, can, t, update, his, fac...","[upset, update, facebook, texting, might, cry,..."
2,0,@Kenichan I dived many times for the ball. Man...,i dived many times for the ball managed to sa...,"[i, dived, many, times, for, the, ball, manage...","[dived, many, times, ball, managed, save, rest..."
3,0,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,"[my, whole, body, feels, itchy, and, like, its...","[whole, body, feels, itchy, like, fire]"
4,0,"@nationwideclass no, it's not behaving at all....",no it s not behaving at all i m mad why am i ...,"[no, it, s, not, behaving, at, all, i, m, mad,...","[behaving, mad, see]"


In [5]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(data['filtered_tokens'], data['label_encoded'], test_size=0.2, random_state=42)

KeyError: 'label_encoded'

then we do bigram analysos

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Create an N-gram model (e.g., bigrams)
ngram_vectorizer = CountVectorizer(ngram_range=(1, 2))
X_train_ngrams = ngram_vectorizer.fit_transform(X_train)
X_test_ngrams = ngram_vectorizer.transform(X_test)


use a multinomial nb classifier

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Train a classifier (e.g., Multinomial Naive Bayes)
classifier = MultinomialNB()
classifier.fit(X_train_ngrams, y_train)

# Make predictions
y_pred = classifier.predict(X_test_ngrams)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred, target_names=le.classes_)

print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{classification_rep}')


In [ ]:
#test